# 1

In [1]:
import psycopg2

# Подключение к PostgreSQL
def connect_to_db():
    conn = psycopg2.connect(
        dbname="python", 
        user="postgres", 
        password="postgres", 
        host="127.0.0.1", 
        port="5432"
    )
    return conn

conn = connect_to_db()

# 2

In [2]:
# Добавление одной строки в таблицу Districts
def add_district_one(conn, code, name, region, head_of_admin):
    cursor = conn.cursor()
    query = """
        INSERT INTO Districts (Code, Name, Region, HeadOfAdministration)
        VALUES (%s, %s, %s, %s)
    """
    cursor.execute(query, (code, name, region, head_of_admin))
    conn.commit()

# Добавление нескольких строк в таблицу Districts
def add_districts_many(conn, districts_data):
    cursor = conn.cursor()
    query = """
        INSERT INTO Districts (Code, Name, Region, HeadOfAdministration)
        VALUES (%s, %s, %s, %s)
    """
    cursor.executemany(query, districts_data)
    conn.commit()

# Аналогичные функции для Cultures и CropYields:
def add_culture_one(conn, code, name, family):
    cursor = conn.cursor()
    query = """
        INSERT INTO Cultures (Code, Name, Family)
        VALUES (%s, %s, %s)
    """
    cursor.execute(query, (code, name, family))
    conn.commit()

def add_cultures_many(conn, cultures_data):
    cursor = conn.cursor()
    query = """
        INSERT INTO Cultures (Code, Name, Family)
        VALUES (%s, %s, %s)
    """
    cursor.executemany(query, cultures_data)
    conn.commit()

def add_crop_yield_one(conn, district, culture, yield_val, year):
    cursor = conn.cursor()
    query = """
        INSERT INTO CropYields (District, Culture, Yield, Year)
        VALUES (%s, %s, %s, %s)
    """
    cursor.execute(query, (district, culture, yield_val, year))
    conn.commit()

def add_crop_yields_many(conn, yields_data):
    cursor = conn.cursor()
    query = """
        INSERT INTO CropYields (District, Culture, Yield, Year)
        VALUES (%s, %s, %s, %s)
    """
    cursor.executemany(query, yields_data)
    conn.commit()




проверка

In [ ]:
conn = connect_to_db()
# Добавление данных в Districts
add_district_one(conn, 1, 'Saraevsky', 'Ryazan', 'Tolmachov')
districts_data = [
    (2, 'Sasovsky', 'Ryazan', 'Rybin'),
    (3, 'Proński', 'Ryazan', 'Kazakov')
]
add_districts_many(conn, districts_data)

# Добавление данных в Cultures
add_culture_one(conn, 101, 'Wheat', 'Grasses')
cultures_data = [
    (102, 'Potato', 'Nightshades'),
    (103, 'Maize', 'Grasses')
]
add_cultures_many(conn, cultures_data)

# Добавление данных в CropYields
add_crop_yield_one(conn, 1, 101, 9.0, 2018)
yields_data = [
    (1, 101, 16.0, 2019),
    (2, 101, 27.0, 2018)
]
add_crop_yields_many(conn, yields_data)

conn.close()

# 3

In [3]:
def select_all_districts(conn):
    """Выбирает все данные из таблицы Districts"""
    cursor = conn.cursor()
    query = "SELECT * FROM Districts;"
    cursor.execute(query)
    return cursor.fetchall()

def select_districts_by_region(conn, region):
    """Выбирает районы по региону"""
    cursor = conn.cursor()
    query = "SELECT * FROM Districts WHERE Region = %s;"
    cursor.execute(query, (region,))
    return cursor.fetchall()

In [4]:
def select_all_cultures(conn):
    """Выбирает все данные из таблицы Cultures"""
    cursor = conn.cursor()
    query = "SELECT * FROM Cultures;"
    cursor.execute(query)
    return cursor.fetchall()

def select_cultures_by_family(conn, family):
    """Выбирает культуры по семейству"""
    cursor = conn.cursor()
    query = "SELECT * FROM Cultures WHERE Family = %s;"
    cursor.execute(query, (family,))
    return cursor.fetchall()

In [5]:
def select_all_crop_yields(conn):
    """Выбирает все данные из таблицы CropYields"""
    cursor = conn.cursor()
    query = "SELECT * FROM CropYields;"
    cursor.execute(query)
    return cursor.fetchall()

def select_crop_yields_by_year(conn, year):
    """Выбирает урожайность за определенный год"""
    cursor = conn.cursor()
    query = "SELECT * FROM CropYields WHERE Year = %s;"
    cursor.execute(query, (year,))
    return cursor.fetchall()




In [6]:
def select_crop_yields_with_district_and_culture(conn, district_name, culture_name):
    """
    Выбирает данные о урожайности с названиями районов и культур.
    Параметры: 
    - district_name (название района)
    - culture_name (название культуры)
    """
    cursor = conn.cursor()
    query = """
        SELECT 
            d.Name AS DistrictName, 
            c.Name AS CultureName, 
            y.Yield, 
            y.Year
        FROM CropYields y
        JOIN Districts d ON y.District = d.Code
        JOIN Cultures c ON y.Culture = c.Code
        WHERE d.Name = %s AND c.Name = %s;
    """
    cursor.execute(query, (district_name, culture_name))
    return cursor.fetchall()

проверка

In [9]:
# Подключение к базе данных
conn = connect_to_db()

# Пример 1: Получить все районы
districts = select_all_districts(conn)
print("Все районы:", districts)

# Пример 2: Получить районы по региону
ryazan_districts = select_districts_by_region(conn, "Ryazan")
print("Районы Рязанской области:", ryazan_districts)

# Пример 3: Получить урожайность за 2018 год
yields_2018 = select_crop_yields_by_year(conn, 2018)
print("Урожайность 2018:", yields_2018)

# Пример 4: Получить урожайность пшеницы в районе Saraevsky
wheat_yields = select_crop_yields_with_district_and_culture(conn, "Saraevsky", "Wheat")
print("Урожайность пшеницы в Saraevsky:", wheat_yields)

conn.close()

Все районы: [(1, 'Saraevsky', 'Ryazan', 'Tolmachov'), (2, 'Sasovsky', 'Ryazan', 'Rybin'), (3, 'Proński', 'Ryazan', 'Kazakov')]
Районы Рязанской области: [(1, 'Saraevsky', 'Ryazan', 'Tolmachov'), (2, 'Sasovsky', 'Ryazan', 'Rybin'), (3, 'Proński', 'Ryazan', 'Kazakov')]
Урожайность 2018: [(1, 101, Decimal('9.00'), 2018), (2, 101, Decimal('27.00'), 2018)]
Урожайность пшеницы в Saraevsky: [('Saraevsky', 'Wheat', Decimal('9.00'), 2018), ('Saraevsky', 'Wheat', Decimal('16.00'), 2019)]


# 4

In [7]:
def update_district(conn, code, name, region, head_of_admin):
    """
    Обновляет данные в таблице Districts по полю Code.
    Параметры: 
    - code (уникальный идентификатор района)
    - name (новое название района)
    - region (новый регион)
    - head_of_admin (новый руководитель администрации)
    """
    cursor = conn.cursor()
    query = """
        UPDATE Districts 
        SET Name = %s, Region = %s, HeadOfAdministration = %s 
        WHERE Code = %s;
    """
    cursor.execute(query, (name, region, head_of_admin, code))
    conn.commit()

In [8]:
def update_culture(conn, code, name, family):
    """
    Обновляет данные в таблице Cultures по полю Code.
    Параметры: 
    - code (уникальный идентификатор культуры)
    - name (новое название культуры)
    - family (новое семейство культуры)
    """
    cursor = conn.cursor()
    query = """
        UPDATE Cultures 
        SET Name = %s, Family = %s 
        WHERE Code = %s;
    """
    cursor.execute(query, (name, family, code))
    conn.commit()

In [9]:
def update_crop_yield(conn, district, culture, year, new_yield, new_year=None):
    """
    Обновляет данные в таблице CropYields по полям District, Culture и Year.
    Параметры: 
    - district (код района)
    - culture (код культуры)
    - year (год урожая)
    - new_yield (новое значение урожайности)
    - new_year (новый год, если требуется изменить первичный ключ)
    """
    cursor = conn.cursor()
    
    # Если требуется изменить Year, обновляем составной первичный ключ
    if new_year is not None:
        query = """
            UPDATE CropYields 
            SET Yield = %s, Year = %s 
            WHERE District = %s AND Culture = %s AND Year = %s;
        """
        cursor.execute(query, (new_yield, new_year, district, culture, year))
    else:
        query = """
            UPDATE CropYields 
            SET Yield = %s 
            WHERE District = %s AND Culture = %s AND Year = %s;
        """
        cursor.execute(query, (new_yield, district, culture, year))
    
    conn.commit()

проверка

In [11]:
# Подключение к базе данных
conn = connect_to_db()

# Обновление данных в таблице Districts
update_district(conn, 1, "New Saraevsky", "New Ryazan", "New Tolmachov")

# Обновление данных в таблице Cultures
update_culture(conn, 101, "New Wheat", "New Grasses")

# Обновление данных в таблице CropYields
update_crop_yield(conn, 1, 101, 2018, 10.0)  # Изменение только Yield
update_crop_yield(conn, 1, 101, 2018, 10.0, 2020)  # Изменение Yield и Year

conn.close()

# 5

In [10]:
def delete_district_by_code(conn, code):
    """
    Удаляет район по его коду (Code).
    Параметр: 
    - code (уникальный идентификатор района)
    """
    cursor = conn.cursor()
    query = "DELETE FROM Districts WHERE Code = %s;"
    cursor.execute(query, (code,))
    conn.commit()

def delete_all_districts(conn):
    """
    Удаляет все данные из таблицы Districts.
    """
    cursor = conn.cursor()
    query = "DELETE FROM Districts;" 
    cursor.execute(query)
    conn.commit()

In [11]:
def delete_culture_by_code(conn, code):
    """
    Удаляет культуру по её коду (Code).
    Параметр: 
    - code (уникальный идентификатор культуры)
    """
    cursor = conn.cursor()
    query = "DELETE FROM Cultures WHERE Code = %s;"
    cursor.execute(query, (code,))
    conn.commit()

def delete_all_cultures(conn):
    """
    Удаляет все данные из таблицы Cultures.
    """
    cursor = conn.cursor()
    query = "DELETE FROM Cultures;" 
    cursor.execute(query)
    conn.commit()

In [12]:
def delete_crop_yield_by_year(conn, year):
    """
    Удаляет данные об урожайности за определенный год.
    Параметр: 
    - year (год урожая)
    """
    cursor = conn.cursor()
    query = "DELETE FROM CropYields WHERE Year = %s;"
    cursor.execute(query, (year,))
    conn.commit()

def delete_all_crop_yields(conn):
    """
    Удаляет все данные из таблицы CropYields.
    """
    cursor = conn.cursor()
    query = "DELETE FROM CropYields;" 
    cursor.execute(query)
    conn.commit()

In [14]:
# Подключение к базе данных
conn = connect_to_db()

# Удаление конкретного района
delete_district_by_code(conn, 1)

# Удаление всех данных из CropYields
delete_all_crop_yields(conn)

# Удаление культуры с кодом 101
delete_culture_by_code(conn, 101)

conn.close()